<a href="https://colab.research.google.com/github/freyaarby/Freepass-DS-BCC-2025/blob/main/DS_Putu_Adelia_Devani_Ardiana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("arxiv_ml.csv")

print(df.head())

In [ ]:
#Data Processing
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#Melakukan Preprocessing
def preprocess_text(text):

    #Penghilangan Lowercasing
    text = text.lower()

    #Penghilangan Karakter Spesial
    text = re.sub(r'\W', ' ', text)

    #Tokenisasi
    tokens = word_tokenize (text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

#Menerapkan preprocessing ke dataset
df["processed_abstract"] = df["abstract"].apply(preprocess_text)

print(df.head())


In [ ]:
#Membangun Sistem Retrieval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Melakukan Inisialisasi Model TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["processed_abstract"])

#Melakukan Retrieval
def retrieve_documents(query, top_k=5):
    query_vec = vectorizer.transform([preprocess_text(query)])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return df.iloc[top_indices][["title", "abstract"]]

#Analisis Contoh pertanyaan
query = "What is the latest development in deep learning?"
retrieved_docs = retrieve_documents(query)
print(retrieved_docs)

query1 = "What is deep learning?"
retrieved_docs1 = retrieve_documents(query1)
print(retrieved_docs1)

query2 = "What are the advantages of Transformer models compared to RNNs and CNNs in Machine Learning?"
retrieved_docs2 = retrieve_documents(query2)
print(retrieved_docs2)

query3 = "How does the Attention Mechanism work in modern NLP models?"
retrieved_docs3 = retrieve_documents(query3)
print(retrieved_docs3)

query4 = "What are the key differences between supervised, unsupervised, and reinforcement learning?"
retrieved_docs4 = retrieve_documents(query4)
print(retrieved_docs4)

query5 = "How is fine-tuning applied to pretrained models like BERT or GPT?"
retrieved_docs5 = retrieve_documents(query5)
print(retrieved_docs5)


In [ ]:
#Membangun Sistem Generasi Jawaban dengan T5
from transformers import T5ForConditionalGeneration, T5Tokenizer

#Load model dan tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

#Melakukan Generasi
def generate_answer(question, context):
    input_text = f"question: {question}  context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = model.generate(input_ids)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

#Analisis Contoh Penggunaan
question1 = "What is deep learning?"
context1 = retrieved_docs.iloc[0]["abstract"]
answer1 = generate_answer(question1, context1)
print(answer1)

question2 = "What are the advantages of Transformer models compared to RNNs and CNNs in Machine Learning?"
context2 = retrieved_docs.iloc[0]["abstract"]
answer2 = generate_answer(question2, context2)
print(answer2)

question3 = "How does the Attention Mechanism work in modern NLP models?"
context3 = retrieved_docs.iloc[0]["abstract"]
answer3 = generate_answer(question3, context3)
print(answer3)

question4 = "What are the key differences between supervised, unsupervised, and reinforcement learning?"
context4 = retrieved_docs.iloc[0]["abstract"]
answer4 = generate_answer(question4, context4)
print(answer4)

question5 = "How is fine-tuning applied to pretrained models like BERT or GPT?"
context5 = retrieved_docs.iloc[0]["abstract"]
answer5 = generate_answer(question5, context5)
print(answer5)

In [ ]:
#Evaluasi Model Generasi (Response Relevancy)
from sklearn.metrics import recall_score

#Simulasi y_true (ground truth) dan y_pred (hasil sistem)
y_true = [1, 1, 0, 1, 0]  #Artikel relevan atau tidak (1=relevan, 0=tidak)
y_pred = [1, 1, 0, 0, 1]

recall_at_k = recall_score(y_true, y_pred, average='macro')
print(f'Recall@K: {recall_at_k}')


In [ ]:
#Evaluasi Model Retrieval (Context Precision)
#Menghitung BLEU
bleu_score = sentence_bleu(reference, candidate)
print(f'BLEU Score: {bleu_score}')

#Menghitung ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference[0], candidate[0])
print(f'ROUGE Scores: {scores}')